<a href="https://colab.research.google.com/github/dmcabrera/genai-cobranza-demo/blob/main/ENTEL_Demo_Diferencias_de_Facturaci%C3%B3n_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2024 Deloitte
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [9]:
# Instalar paquetes
!pip install -q --upgrade google-cloud-aiplatform
!pip install -q markdown

In [2]:
# Reiniciar runtime
import IPython
app = IPython.Application.instance().kernel.do_shutdown(True)

In [1]:
# Autenticar colab
from google.colab import auth
auth.authenticate_user()

In [2]:
# Constantes
PROJECT_ID = "dtt-genai-demos" # TODO: Replace with your project ID
LOCATION = "us-central1" # TODO: Replace with your deployment location

In [3]:
# Importo módulos
import vertexai
from vertexai.generative_models import GenerativeModel
import vertexai.generative_models as generative_models

In [4]:
# Configuración de generación del modelo
GENERATION_CONFIG = {
    "max_output_tokens": 8192,
    "temperature": 0.3,
    "top_p": 0.95,
}

# Safe settings del modelo
SAFETY_SETTINGS = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [5]:
llm = GenerativeModel(
    "gemini-1.5-flash",
    system_instruction="""
Eres Entel Cobranza, un asistente especializado en responder preguntas asociadas \
a la facturación de los clientes segun el siguiente contexto:

Contexto:
{
  "apellidos": "Gonzalez Perez",
  "nombres" : "Juan Manuel",
  "rut" : "12345678-9",
  "factura_anterior" : {
    "fecha" : "202407",
    "numero" : "4433556677",
    "monto" : 65000,
    "detalle" : [
      {
        "tipo_servicio" : "movil",
        "detalle" : "5622334455",
        "monto" : 35000
      },
      {
        "tipo_servicio" : "television",
        "detalle" : "plan_gold",
        "monto" : 30000
      }
    ]
  },
  "factura_actual" : {
    "fecha" : "202408",
    "numero" : "4433588232",
    "monto" : 71000,
    "detalle" : [
      {
        "tipo_servicio" : "movil",
        "detalle" : "5622334455",
        "monto" : 35000
      },
      {
        "tipo_servicio" : "television",
        "detalle" : "plan_gold",
        "monto" : 30000
      },
      {
        "servicio" : "tv_ondemand",
        "detalle" : "spider man 3",
        "monto" : 5000
      }
    ]
  }
}
            """,
            generation_config=GENERATION_CONFIG,
            safety_settings=SAFETY_SETTINGS
        )

In [20]:
from IPython.display import display, Markdown


# Genero texto
def generate(pregunta):
  responses = llm.generate_content(
    pregunta,
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.2,
        "top_p": 1
    },
    stream=False,
  )

  display(Markdown(responses.candidates[0].text))

In [21]:
generate("por qué ne están cobrando más este mes?")

Estimado Juan Manuel Gonzalez Perez, le informamos que el monto de su factura del mes de agosto (número 4433588232) es de $71.000, lo que representa un aumento de $6.000 con respecto a la factura anterior. 

Este aumento se debe a la incorporación de un nuevo servicio en su factura: **tv_ondemand** por el detalle "spider man 3" con un valor de $5.000.

El resto de los servicios, como su plan móvil 5622334455 y el plan de televisión "plan_gold", mantienen el mismo valor que el mes anterior.

Si tiene alguna duda o necesita mayor información, no dude en contactarnos.


In [22]:
generate("cuánto me están cobrando por el servicio de televisión?")

En su factura actual, correspondiente al mes de 202408, el servicio de televisión le está costando $30.000. 

¿Le puedo ayudar con algo más? 
